# Check for FH Mutations in the cases

In [1]:
import pandas as pd 

# Read in the annotated dataframe
pe2_df = pd.read_csv('data/annotated-rna_kirp-Pe2_20200814.csv')
pe2_df

,gene_id,gene_name,FHgrt11.8_P-e.2_TCGA-KIRC_PrimaryTumor_male_blackorafricanamerican_1_htseq.counts_735_TCGA-KIRC_TCGA-A3-3313_2671dda7-926b-4b95-9b0c-5ebe9a40f0c2,FHgrt11.8_P-e.2_TCGA-KIRC_PrimaryTumor_male_blackorafricanamerican_2_htseq.counts_--_TCGA-KIRC_TCGA-AK-3456_7b71cfe6-4fdd-4c2b-9b46-d7589c896174,FHleq11.8_P-e.2_TCGA-KIRC_PrimaryTumor_female_blackorafricanamerican_1_htseq.counts_51_TCGA-KIRC_TCGA-B2-4098_6d9015d6-2470-4ebb-8e07-360cde9ab13f,FHgrt11.8_P-e.2_TCGA-KIRC_SolidTissueNormal_female_white_4_htseq.counts_552_TCGA-KIRC_TCGA-CJ-5681_48a60ec9-abe8-4c67-907d-eefd97b7a107,FHgrt11.8_P-e.2_TCGA-KIRC_PrimaryTumor_female_white_4_htseq.counts_552_TCGA-KIRC_TCGA-CJ-5681_c3641a4d-1af6-474a-9313-4d1877b69bd4,FHgrt11.8_P-e.2_TCGA-KIRC_PrimaryTumor_female_white_1_htseq.counts_--_TCGA-KIRC_TCGA-B4-5835_86c8062c-ae80-45c6-a35b-591043a9649d,FHgrt11.8_P-e.2_TCGA-KIRC_PrimaryTumor_female_blackorafricanamerican_1_htseq.counts_--_TCGA-KIRC_TCGA-B8-A54E_e29ee40c-a51c-490e-b29e-d0d1c12de8ac,FHgrt11.8_P-e.2_TCGA-KIRC_PrimaryTumor_female_blackorafricanamerican_1_htseq.counts_--_TCGA-KIRC_TCGA-6D-AA2E_651d7063-0803-459e-b03a-b2c81ddf87a5,...,FHleq11.8_P-e.2_TCGA-KIRP_PrimaryTumor_male_white_1_htseq.counts_--_TCGA-KIRP_TCGA-Y8-A8S1_113e20dd-1cea-4ae6-95b1-8c17f5b36f88,FHgrt11.8_P-e.2_TCGA-KIRP_PrimaryTumor_male_asian_3_htseq.counts_--_TCGA-KIRP_TCGA-F9-A97G_1ae69dce-655a-4175-b175-d513e8a069cc,FHgrt11.8_P-e.2_TCGA-KIRP_PrimaryTumor_male_white_1_htseq.counts_--_TCGA-KIRP_TCGA-2Z-A9JD_c02afc0a-11c5-4fcc-8986-287d79875b19,FHgrt11.8_P-e.2_TCGA-KIRP_PrimaryTumor_female_blackorafricanamerican_1_htseq.counts_--_TCGA-KIRP_TCGA-2Z-A9JO_68296760-4635-4a8e-bafb-839aef7b2b75,FHleq11.8_P-e.2_TCGA-KIRP_PrimaryTumor_female_white_1_htseq.counts_589_TCGA-KIRP_TCGA-5P-A9K9_8e994ca2-33c6-4ff3-9028-caaee46b93c9,FHgrt11.8_P-e.2_TCGA-KIRP_PrimaryTumor_female_white_1_htseq.counts_--_TCGA-KIRP_TCGA-5P-A9KC_1e43f447-863c-4de8-a607-0fd210444de7,FHleq11.8_P-e.2_TCGA-KIRP_PrimaryTumor_male_white_1_htseq.counts_--_TCGA-KIRP_TCGA-P4-AAVO_49be8b02-bb76-49a4-869b-5995a59ec10c,FHgrt11.8_P-e.2_TCGA-KIRP_PrimaryTumor_male_blackorafricanamerican_3_htseq.counts_2624_TCGA-KIRP_TCGA-UZ-A9PQ_a8813427-dfd6-416a-8a06-ffcad7cd4e16,FHgrt11.8_P-e.2_TCGA-KIRP_PrimaryTumor_female_white_1_htseq.counts_--_TCGA-KIRP_TCGA-UZ-A9Q1_af77b60e-93d7-45af-bfd9-879078338349,FHgrt11.8_P-e.2_TCGA-KIRP_PrimaryTumor_female_white_1_htseq.counts_--_TCGA-KIRP_TCGA-WN-AB4C_ee82feda-b712-4432-91f6-3dfa9d5dccbb
0,ENSG00000000003.13,TSPAN6,3225,4059,2020,4104,2974,10472,2379,2332,...,1804,3918,2006,2830,4114,3125,3467,2603,4053,2079
1,ENSG00000000005.5,TNMD,0,15,0,14,0,12,0,1,...,0,4,0,3,27,1,1,1,6,0
2,ENSG00000000419.11,DPM1,1490,985,958,1420,1472,1726,1927,1107,...,695,1599,1153,1238,1445,1272,1011,1609,1258,449
3,ENSG00000000457.12,SCYL3,832,547,480,625,829,780,757,455,...,175,656,415,313,403,635,150,882,390,168
4,ENSG00000000460.15,C1orf112,258,125,131,272,197,521,903,126,...,61,143,91,76,284,167,36,203,83,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56535,ENSG00000281904.1,AC233263.6,0,0,0,0,0,0,0,1,...,0,0,0,0,0,3,0,8,0,0
56536,ENSG00000281909.1,HERC2P7,1,0,2,0,1,0,0,0,...,0,3,1,0,1,0,0,2,0,0
56537,ENSG00000281910.1,SNORA50A,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
56538,ENSG00000281912.1,LINC01144,112,25,39,70,344,42,36,99,...,50,18,54,196,47,35,48,31,65,135


In [6]:
## Need to get the case IDs and see if they have mutations
case_ids = list(set([c.split('_')[-2] for c in pe2_df.columns  if len(c.split('_')) > 2]))
case_ids

['TCGA-G7-7501',
 'TCGA-BQ-5875',
 'TCGA-MH-A855',
 'TCGA-B9-A69E',
 'TCGA-BQ-5876',
 'TCGA-GL-A9DE',
 'TCGA-SX-A7SM',
 'TCGA-T7-A92I',
 'TCGA-UZ-A9PQ',
 'TCGA-SX-A7SO',
 'TCGA-AK-3456',
 'TCGA-BQ-7058',
 'TCGA-MH-A560',
 'TCGA-BQ-5890',
 'TCGA-2Z-A9JO',
 'TCGA-F9-A97G',
 'TCGA-B1-A656',
 'TCGA-P4-AAVO',
 'TCGA-5P-A9K9',
 'TCGA-DZ-6131',
 'TCGA-G7-6796',
 'TCGA-B9-A8YI',
 'TCGA-2Z-A9JD',
 'TCGA-CJ-5681',
 'TCGA-6D-AA2E',
 'TCGA-GL-A4EM',
 'TCGA-B4-5835',
 'TCGA-A4-A57E',
 'TCGA-4A-A93Y',
 'TCGA-5P-A9KC',
 'TCGA-A4-7585',
 'TCGA-B8-A54E',
 'TCGA-B1-5398',
 'TCGA-B1-A655',
 'TCGA-BQ-7044',
 'TCGA-BQ-7061',
 'TCGA-B2-4098',
 'TCGA-J7-8537',
 'TCGA-BQ-5884',
 'TCGA-BQ-5885',
 'TCGA-BQ-5882',
 'TCGA-UZ-A9Q1',
 'TCGA-BQ-5887',
 'TCGA-Y8-A8S1',
 'TCGA-IZ-8195',
 'TCGA-A3-3313',
 'TCGA-WN-AB4C',
 'TCGA-BQ-5880']

In [7]:
# Download mutation data
# Here we want to download the mutation files
from scidat.download import Download
from sciutil import SciUtil

supp_dir = f'supps/' # Where you need to put the GDC client
data_dir = ''
u = SciUtil()

download_mutation = True
if download_mutation:

    download = Download(f'{data_dir}data/annotated-rna_kirp-Pe2_20200814.csv', 
                        f'{data_dir}mutations/', 
                        f'{data_dir}mutations/', 
                        f'{supp_dir}gdc-client', 
                        max_cnt=1)
    download.download_data_using_api(case_ids, 'mutation')
else:
    u.dp(["You said you already have your mutations downloaded, if this isn't the case you'll need to do this!"])


--------------------------------------------------------------------------------
Num cases with mutations: 	43	
Num cases without mutations: 	5	

Cases: 
	TCGA-UZ-A9PQ
TCGA-AK-3456
TCGA-P4-AAVO
TCGA-5P-A9K9
TCGA-B2-4098	
--------------------------------------------------------------------------------


In [13]:
# ssm.genomic_dna_change
import os
mutation_dir ='mutations/'
mutation_files = os.listdir(mutation_dir)
gene_id = 'ssm.consequence.0.transcript.gene.symbol'

genes = ['FH', 'HIRA', 'MTOR', 'PTEN', 'VHL', 'PBRM1', 'KDM5C', 'CUBN', 'USH2A', 'SETD2', 'BAP1', 'MUC16']
for gene in genes:
    mutations = []
    cases = []
    change = 'ssm.genomic_dna_change'
    for f in mutation_files:
        df = pd.read_csv(f'{mutation_dir}{f}', sep='\t')
        df = df[df[gene_id] == gene]
        if len(df) > 0:
            mutations.append(df['ssm.genomic_dna_change'].values[0])
            cases.append(f.split('_')[1])
    u.dp([gene, "Number of cases with mutation:", len(mutations), "\n Mutations:", mutations, "\nCase IDs:", cases])

--------------------------------------------------------------------------------
       FH	Number of cases with mutation:	0	
 Mutations:	[]	
Case IDs:	[]	       
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
      HIRA	Number of cases with mutation:	0	
 Mutations:	[]	
Case IDs:	[]	      
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
MTOR	Number of cases with mutation:	2	
 Mutations:	['chr1:g.11210832G>A', 'chr1:g.11247733C>A']	
Case IDs:	['TCGA-GL-A4EM', 'TCGA-BQ-7061']	
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
PTEN	Number of cases with mutation:	1	
 Mutations:	['chr10:g.87894026T>A']	
Case IDs:	['TCGA-UZ-A9Q1']	
--------------------------

In [14]:
c_bio_portal_mutations = ['TCGA-GL-7966', 'TCGA-F9-A4JJ', 'TCGA-G7-6793', 'TCGA-B9-A5W9']
print(set(c_bio_portal_mutations) & set(case_ids))

set()


In [ ]:
## Read in and look at what they were